<a href="https://colab.research.google.com/github/Zelechos/Generador-Css/blob/main/model/GeneradorCSS_v0.2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Proyecto Generador de CSS [v0.2.0]**

---



# Clonamos nuestro repositorio en donde se encuentra nuestro dataset con este comando

In [9]:
!git clone https://github.com/Zelechos/Generador-Css.git

fatal: destination path 'Generador-Css' already exists and is not an empty directory.


### en este caso no es necesario ya que tenemos el dataset en github por ende tenemos que trabajar con el dataset que esta en formato json

Intalamos la libreria trasformers datasets

In [6]:
! pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.7 MB/s eta 0:00:00


### Cargamos nuestro DATASET


In [21]:
from datasets import load_dataset


data_files = {
    "train": "/content/Generador-Css/dataset/STYLES.json",
    "validation": "/content/Generador-Css/dataset/TEST.json",
    "test": "/content/Generador-Css/dataset/TEST.json"
}

dataset = load_dataset("json", data_files=data_files, field='styles')

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

# Analisamos el Dataset

In [23]:
print(dataset.data)

{'train': MemoryMappedTable
style: string
selector: string
----
style: [["body { overflow:hidden ; background:rgb(25,35,125) } ","div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ","div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ","div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ","div.drop-container { position:absolute ; top:0 ; right:0 ; bottom:0 ; left:0 ; margin:auto ; height:200px ; width:200px } ",...,".image-rotate { overflow:hidden ; margin:8px ; min-width:240px ; max-width:320px ; width:100% ; } .image-rotate img { transition:all 0.3s ; box-sizing:border-box ; max-width:100% ; } .image-rotate:hover img { transform:scale(1.3) rotate(5deg) ; } ",".image-rotate { overflow:hidden ; margin:8px ; min-width:240px ; max-width:320px ; width:100% ; } .image-rotate img 

### Cargando nuestro modelo Pre-entrenado GPT-2 XL

In [1]:
from transformers import GPT2Tokenizer, TFGPT2Model

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
model = TFGPT2Model.from_pretrained('gpt2-xl')

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='gpt2-xl')
set_seed(42)
generator("The man worked as a", max_length=10, num_return_sequences=5)

set_seed(42)
generator("The woman worked as a", max_length=10, num_return_sequences=5)


Analisis del Modelo GTP-2 XL

**Experimento 001**
- Cambiamos el parametro max_length a 50
que dadas las observaciones define la longitud de palabras maximas que puede tener la sentencia a generar
- El parametro num_return_sequences = 1

In [6]:
set_seed(42)
generator("Alan Turing was a", max_length=50, num_return_sequences=1)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'Alan Turing was a brilliant mathematician, but you can\'t beat a human being in a chess match with any of the chess engines out there. So for computer chess, he said, "You are only ever going to beat a human being in a game'}]

**Experimento 002**
- Cambiamos el parametro max_length a 2
que dadas las observaciones define la longitud de palabras maximas que puede tener la sentencia a generar
- Tambien Cambiamos el parametro num_return_sequences a 3
que dadas las observaciones es la cantidad de sentencias que genera el modelo considerando la longitud de palabras antes definida

*Se observo que al poner el parametro de max_length = 2 presenta un error por la longitud del input*

In [7]:
generator("a Transformer in AI is", max_length=2, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ValueError: Input length of input_ids is 2, but `max_length` is set to 2. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.

**Experimento 003**
- Cambiamos el parametro max_length a 7
que dadas las observaciones define la longitud de palabras maximas que puede tener la sentencia a generar incluyendo el input
- Tambien Cambiamos el parametro num_return_sequences a 3
que dadas las observaciones es la cantidad de sentencias que genera el modelo considerando la longitud de palabras antes definida

Se observo que con el parametro max_length = 2 hay una error dado que el parametro max_length debe superar en longitud de input_ids es N entonces el parametro max_length siempre tiene que ser almenos N + 1 o mayor que.

In [11]:
generator("a Transformer in AI is", max_length=10, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'a Transformer in AI is the same as that'},
 {'generated_text': 'a Transformer in AI is a powerful engine ('},
 {'generated_text': 'a Transformer in AI is more than worth it'}]

**Experimento 004**
- Definimos el parametro de la semilla en 30 para observar que sucede


In [12]:
set_seed(30)
generator("a Artificial Neural Network is", max_length=15, num_return_sequences=3)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'a Artificial Neural Network is a machine learning algorithm that combines a number of techniques'},
 {'generated_text': 'a Artificial Neural Network is a program capable of learning and recognizing symbols and patterns'},
 {'generated_text': 'a Artificial Neural Network is trained on a large number of digitized music examples'}]

# Aplicar Transfer Learning de GPT-2 XL a GeneradorCSS

In [ ]:
from transformers import GPT2Tokenizer, TFGPT2Model

tokenizer = GPT2Tokenizer.from_pretrained('gpt2-xl')
model = TFGPT2Model.from_pretrained('gpt2-xl')

text = "Replace me by any text you'd like."
encoded_input = tokenizer(text, return_tensors='tf')
output = model(encoded_input)

Creamos la funcionr para tokenizar el dataset

In [2]:
# dataset ubication => data_styles = data["styles"]
def tokenize_function(examples):
  return tokenizer(examples["styles"], padding="max_length", truncation=True)

Tokenizamos nuestro dataset

In [3]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)

AttributeError: 'dict' object has no attribute 'map'